In [1]:
import pandas as pd
import numpy as np
from ipynb.fs.full.interp_data import interpolate_scenario_data

Extract data from AR6 database
- "Capacity|Electricity" GW
- "Final Energy|Electricity" EJ/yr

In [2]:
# load data
ar6_world = pd.read_csv("../../data/AR6_data/1668008174375-AR6_Scenarios_Database_R10_regions_v1.1.csv/AR6_Scenarios_Database_R10_regions_v1.1.csv")
ar6_scen = pd.read_excel("../../data/AR6_data/AR6_Scenarios_Database_World_v1.1/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx",sheet_name = "meta_Ch3vetted_withclimate")

In [3]:
ar6_scen_123 = ar6_scen[ar6_scen["Category"].isin(["C1","C2","C3"])]

In [8]:
#var = "Capacity|Electricity"
#value_name = "GW"
#file_name = "AR6_Electricity_GW_R10_capacity.csv"

var_list = ["Secondary Energy|Electricity","Secondary Energy|Electricity|Coal|w/o CCS",
           "Secondary Energy|Electricity|Gas|w/o CCS","Secondary Energy|Electricity|Solar",
            "Secondary Energy|Electricity|Wind"]
value_name_list  = ["EJ/yr","EJ/yr","EJ/yr","EJ/yr","EJ/yr"]
file_name_list = ["AR6_SecEnergy_el_R10.csv","AR6_coal_el_R10.csv","AR6_gas_el_R10.csv",
                 "AR6_solar_el_R10.csv","AR6_wind_el_R10.csv"]

for i in np.arange(len(var_list)): 
    var = var_list[i]
    value_name = value_name_list[i]
    file_name = file_name_list[i]
    ar6_world_coal = ar6_world.query("Variable==@var")
    ar6_world_coal = interpolate_scenario_data(ar6_world_coal)
    ar6_world_coal = pd.merge(ar6_world_coal,ar6_scen_123[["Model","Scenario","Category"]],
                                  left_on=["Model","Scenario"],right_on=["Model","Scenario"])
    ar6_world_coal_melt = pd.melt(ar6_world_coal,id_vars=["Model","Scenario","Category","Region"],value_vars=ar6_world_coal.columns[5:-1],var_name="Year",value_name=value_name).dropna()
    ar6_world_coal_melt["Year"] = ar6_world_coal_melt["Year"].astype(int)
    ar6_world_coal_melt = ar6_world_coal_melt.sort_values("Category")
    ar6_world_coal_melt.set_index("Model").to_csv(file_name)
ar6_world_coal_melt